In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("/content/dfs.csv")

In [ ]:
pip install lazypredict

In [ ]:
import lazypredict

In [ ]:
from sklearn.model_selection import train_test_split

X= df.drop("rent",axis=1)
y=df["rent"]

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=.3,random_state =123)

In [ ]:
from lazypredict.Supervised import LazyRegressor

regressor = LazyRegressor(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = regressor.fit(X_train, X_test, y_train, y_test)

# Print the performance of each model
print(models)

In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from math import sqrt


from sklearn.model_selection import cross_val_score

# Using the best parameters from the previous LGBM model
best_lgbm_params = {
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'n_estimators': 300,
    'num_leaves': 31
}

# Initialize LGBM with the best parameters and regularization
model = lgb.LGBMRegressor(**best_lgbm_params, random_state=123, reg_alpha=0.1, reg_lambda=0.1)

# Perform cross-validation to get the RMSE
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_squared_error', cv=5)

# Calculate the average RMSE across all folds
rmse_reg = sqrt(-scores.mean())

# Fit the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred_reg = model.predict(X_test)

# Calculate the RMSE on the test set
rmse_test_reg = sqrt(mean_squared_error(y_test, y_pred_reg))

# Output the RMSE from cross-validation and the test set
print('Cross-validated RMSE with regularization:', rmse_reg)
print('Test set RMSE with regularization:', rmse_test_reg)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002100 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 902
[LightGBM] [Info] Number of data points in the train set: 10880, number of used features: 18
[LightGBM] [Info] Start training from score 19124.431342
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001824 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 903
[LightGBM] [Info] Number of data points in the train set: 10880, number of used features: 18
[LightGBM] [Info] Start training from score 19182.322610
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002069 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, y_pred_reg)

print(f"R-squared score: {r2}")

R-squared score: 0.8215602140973817


In [ ]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)